In [5]:
import pandas as pd

df = pd.read_csv("energy_consumption_queries.csv")

df.to_excel("energy_temp.xlsx")

In [14]:
from agent.configuration import Configuration
from agent.utils import load_chat_model
import pandas as pd
import yaml

c:\Users\scerda\Documents\langgraph-postgres-energy-data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
model = load_chat_model("ollama/testing")

In [16]:
from agent.state import QueryOutput

In [37]:
pregunta = "Cual es el tipo de edificio con mayor consumo en el 2025 y como se compara con el 2024?"

text=f"You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\nYou must output the postgres SQL query that answers the question\n\nDATABASE SCHEMA:\n\nCREATE TABLE smart_buildings.building (\n    cups TEXT PRIMARY KEY,\n    name TEXT NOT NULL,\n    address TEXT,\n    type TEXT\n);\n\nCREATE TABLE smart_buildings.energy_consumption_monthly_metrics (\n\tcups text NULL,\n\tyear_month date NULL,\n\ttotal_consumption_kwh float8 NULL,\n\tavg_daily_consumption_kwh float8 NULL,\n\tstd_daily_consumption_kwh float8 NULL,\n\tdays_in_month int8 NULL,\n\ttotal_consumption_prev_month_kwh float8 NULL,\n\ttotal_consumption_prev_year_same_month_kwh float8 NULL,\n\tdiff_pct_consumption_prev_month float8 NULL,\n\tytd_consumption_kwh numeric(38, 6) NULL,\n\tytd_prev_year_consumption_kwh numeric(38, 6) NULL,\n\tdate_insert timestamp NOT NULL,\n\tdiff_pct_consumption_prev_year_same_month float8 NULL\n);\nCREATE TABLE smart_buildings.energy_consumption_weekly_metrics (\n    cups TEXT NOT NULL,\n    week_start DATE NOT NULL,\n    total_consumption_kwh DOUBLE PRECISION,\n    daily_consumption_kwh DOUBLE PRECISION,\n    total_consumption_prev_week_kwh DOUBLE PRECISION,\n    diff_pct_consumption_prev_week DOUBLE PRECISION,\n    std_daily_consumption_kwh DOUBLE PRECISION,\n    date_insert TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n    PRIMARY KEY (cups, week_start),\n    FOREIGN KEY (cups) REFERENCES smart_buildings.building(cups)\n);\n\nBuilding types:\n['Administración', 'Educación', 'Comercio', 'Punto Limpio', 'Casal/Centro Cívico', 'Cultura y Ocio', 'Restauración', 'Salud y Servicios Sociales', 'Bienestar Social', 'Mercado', 'Parque', 'Industrial', 'Centros Deportivos', 'Parking', 'Policia', 'Cementerio', 'Protección Civil']\n\nReturn **only** the SQL query—no additional explanation or formatting:\n\n{pregunta}\n"

response=model.invoke(text)

In [38]:
print(response.content)

```sql
SELECT b.type,
       SUM(e.total_consumption_kwh) AS total_consumption_kwh,
       AVG(e.total_consumption_kwh) AS avg_monthly_consumption_kwh,
       SUM(e.total_consumption_prev_year_same_month_kwh) AS total_consumption_prev_year
FROM smart_buildings.energy_consumption_monthly_metrics e
JOIN smart_buildings.building b ON e.cups = b.cups
WHERE EXTRACT(YEAR
              FROM e.year_month) = 2025
GROUP BY b.type
ORDER BY total_consumption_kwh DESC
LIMIT 1
```


In [17]:
print(response.query)

SELECT b.name AS building_name, 
 m.year_month, 
 m.total_consumption_kwh, 
 m.avg_daily_consumption_kwh, 
 m.total_consumption_prev_month_kwh, 
 m.diff_pct_consumption_prev_month, 
 m.std_daily_consumption_kwh, 
 m.ytd_consumption_kwh 
FROM smart_buildings.building b 
JOIN smart_buildings.energy_consumption_monthly_metrics m ON b.cups = m.cups 
WHERE b.name = 'Torres Norte' 
AND EXTRACT(YEAR  FROM m.year_month) = EXTRACT(YEAR FROM CURRENT_DATE) 
ORDER BY m.year_month DESC 
LIMIT 1;


In [9]:
import pandas as pd
# Parámetros de entrada
excel_file = 'training_generated.csv'

# Lee el Excel
df = pd.read_csv(excel_file,encoding='latin1',sep=";")

In [11]:
df.sql.values

array(["SELECT e.total_consumption_kwh FROM smart_buildings.building b JOIN smart_buildings.energy_consumption_monthly_metrics e ON b.cups = e.cups WHERE b.name = 'Central Park' AND e.year_month = DATE '2024-01-01'",
       "SELECT e.total_consumption_kwh FROM smart_buildings.building b JOIN smart_buildings.energy_consumption_weekly_metrics e ON b.cups = e.cups WHERE b.name = 'Torres Norte' AND EXTRACT(WEEK FROM e.week_start) = 12 AND EXTRACT(YEAR FROM e.week_start) = 2023",
       "SELECT e.avg_daily_consumption_kwh FROM smart_buildings.building b JOIN smart_buildings.energy_consumption_monthly_metrics e ON b.cups = e.cups WHERE b.name = 'Sede Central' AND e.year_month = DATE '2025-03-01'",
       "SELECT e.std_daily_consumption_kwh FROM smart_buildings.building b JOIN smart_buildings.energy_consumption_monthly_metrics e ON b.cups = e.cups WHERE b.name = 'Ayuntamiento' AND e.year_month = DATE '2024-04-01'",
       "SELECT e.year_month, e.total_consumption_kwh FROM smart_buildings.buil

In [13]:
import pandas as pd
import sqlparse

# Parámetros de entrada
excel_file = 'training.csv'

# Lee el Excel
df = pd.read_csv(excel_file,encoding='latin1',sep=";")

def pretty_sql(sql_text):
    if pd.isnull(sql_text):
        return sql_text
    return sqlparse.format(sql_text, reindent=True, keyword_case='upper')

# Aplica el formateo a la columna 'sql'
df['sql_parsed'] = df['sql'].apply(pretty_sql)
df.sql_parsed = df.sql_parsed.str.replace('"','')
df.sql_parsed = "```sql\n"+df.sql_parsed+"\n```"
# Guarda el resultado en un nuevo archivo
output_file = 'training_v2.csv'
df.to_csv(output_file, index=False,encoding="latin1",sep=";")

print(f"Archivo guardado como {output_file}")


Archivo guardado como training_v2.csv


In [3]:
import pandas as pd

df = pd.read_csv("training_generated.csv",sep=";",encoding="latin1")
df

,pregunta,sql,schema,description
0,¿Cuál fue el consumo energético total del edif...,SELECT e.total_consumption_kwh FROM smart_buil...,CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
1,¿Cuántos kWh consumió el edificio Torres Norte...,SELECT e.total_consumption_kwh FROM smart_buil...,CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
2,¿Cuál fue el consumo promedio diario del edifi...,SELECT e.avg_daily_consumption_kwh FROM smart_...,CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
3,¿Me puedes indicar la desviación estándar del ...,SELECT e.std_daily_consumption_kwh FROM smart_...,CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
4,"Cuanto ha sido el consumo de ""Deportivo Unido""...","SELECT e.year_month, e.total_consumption_kwh F...",CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
...,...,...,...,...
105,¿Cuáles son los 8 edificios con mayor consumo ...,"SELECT b.name, SUM(e.total_consumption_kwh) AS...",CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
106,¿Cuál es el top 6 de equipamientos con menor c...,"SELECT b.name, e.year_month, e.diff_pct_consum...",CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
107,¿Cuáles son los 4 equipamientos de Escuelas co...,"SELECT b.name, SUM(e.total_consumption_kwh) AS...",CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."
108,¿Cuál es el consumo total de los centros educa...,"SELECT b.type, SUM(e.total_consumption_kwh) AS...",CREATE TABLE smart_buildings.building (\n c...,"""## Table: building\n\n* **cups**: Universal S..."


In [11]:
import sys
import pandas as pd
import psycopg2
from psycopg2 import sql
import os
def run_queries(csv_path: str, conn_params: dict, show_df: bool = True) -> None:
    # 1) Cargar CSV en DataFrame
    df = pd.read_csv(csv_path, encoding="latin1",sep=";")

    if show_df:
        print("\nPrimeras filas del archivo:")
        print(df.head())                # ‘envía’ el DataFrame creado

    # 2) Conexión
    conn = psycopg2.connect(**conn_params)
    conn.autocommit = True
    cur = conn.cursor()

    # 3) Recorrer y ejecutar cada sentencia
    for idx, query in enumerate(df["sql"], start=1):
        if not isinstance(query, str) or not query.strip():
            continue                    # celda vacía

        try:
            cur.execute(sql.SQL(query))
        except Exception as exc:
            print(f"\n⚠️  Error en fila {idx}:")
            print(query)
            print(f"→ {exc.pgerror or exc}")

    cur.close()
    conn.close()

if __name__ == "__main__":

    CSV_FILE = r"C:\Users\scerda\Documents\langgraph-postgres-energy-data\training_generated.csv"

    # Rellena tus credenciales / base:
    CONN_PARAMS = {
        "host":     os.getenv('DB_HOST'),
        "port":     5432,
        "dbname":   os.getenv('DB_NAME'),
        "user":     os.getenv('DB_USER'),
        "password": os.getenv('DB_PASSWORD')
    }

    run_queries(CSV_FILE, CONN_PARAMS)



Primeras filas del archivo:
                                            pregunta  \
0  ¿Cuál fue el consumo energético total del edif...   
1  ¿Cuántos kWh consumió el edificio Torres Norte...   
2  ¿Cuál fue el consumo promedio diario del edifi...   
3  ¿Me puedes indicar la desviación estándar del ...   
4  Cuanto ha sido el consumo de "Deportivo Unido"...   

                                                 sql  \
0  SELECT e.total_consumption_kwh FROM smart_buil...   
1  SELECT e.total_consumption_kwh FROM smart_buil...   
2  SELECT e.avg_daily_consumption_kwh FROM smart_...   
3  SELECT e.std_daily_consumption_kwh FROM smart_...   
4  SELECT e.year_month, e.total_consumption_kwh F...   

                                              schema  \
0  CREATE TABLE smart_buildings.building (\n    c...   
1  CREATE TABLE smart_buildings.building (\n    c...   
2  CREATE TABLE smart_buildings.building (\n    c...   
3  CREATE TABLE smart_buildings.building (\n    c...   
4  CREATE TABLE s

In [12]:
import os
import pandas as pd
import openai
from copy import deepcopy
from openai import OpenAI
# Configura tu API Key
openai.api_key = os.getenv("OPENAI_API_KEY")  # o pon aquí tu clave directamente: openai.api_key = "tu_api_key"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def paraphrase_question(question, n=3):
    """
    Usa la API de OpenAI para generar n parafraseos de una pregunta dada.
    """
    prompt = f"Parafrasea la siguiente pregunta de forma natural y diferente, manteniendo el mismo sentido:\n\nPregunta: \"{question}\"\n\nGenera {n} preguntas parafraseadas, cada una en una línea separada."
    
    try:
        response = client.responses.create(
            model="gpt-4.1-nano",
            instructions = "Eres un experto en reformulación de preguntas.",
            input = prompt,
        )
        text = response.output_text
        # Asumimos que el modelo devuelve las preguntas parafraseadas una por línea
        paraphrases = [line.strip("- ").strip() for line in text.split("\n") if line.strip()]
        # En caso de que genere más o menos, tomamos solo n
        return paraphrases[:n]
    except Exception as e:
        print(f"Error al parafrasear: {e}")
        return []

def expand_questions(df, question_col='question', paraphrases_per_question=3):
    """
    Dado un DataFrame, genera nuevas filas con preguntas parafraseadas y mantiene las otras columnas iguales.
    """
    new_rows = []
    for _, row in df.iterrows():
        original_question = row[question_col]
        paraphrases = paraphrase_question(original_question, n=paraphrases_per_question)
        
        # Agrega la pregunta original
        new_rows.append(row.to_dict())
        
        # Agrega las parafraseadas, clonando el resto de columnas
        for pq in paraphrases:
            new_row = deepcopy(row.to_dict())
            new_row[question_col] = pq
            new_rows.append(new_row)
    
    return pd.DataFrame(new_rows)

def main():
    input_csv = "training_generated.csv"  # Cambia aquí el nombre de tu archivo
    output_csv = "new_add_questions.csv"
    
    print("Leyendo CSV...")
    df = pd.read_csv(input_csv,encoding="latin1",sep=";")
    
    print("Generando parafraseos...")
    df_expanded = expand_questions(df, question_col='pregunta', paraphrases_per_question=3)
    
    print(f"Guardando resultado en {output_csv} ...")
    df_expanded.to_csv(output_csv, index=False)
    print("Proceso completado.")

if __name__ == "__main__":
    main()


Leyendo CSV...
Generando parafraseos...
Guardando resultado en new_add_questions.csv ...
Proceso completado.


In [5]:
for i in range(0, len(df), 10):
    batch = df.iloc[i:i+10]
    print(batch.sql_parsed)
    break

0    ```sql\nSELECT b.name AS building_name,\n     ...
1    ```sql\nSELECT b.name AS building_name,\n     ...
2    ```sql\nSELECT b.name AS building_name,\n     ...
3    ```sql\nSELECT b.name AS building_name,\n     ...
4    ```sql\nSELECT b.name AS building_name,\n     ...
5    ```sql\nSELECT b.name AS building_name,\n     ...
6    ```sql\nSELECT b.name AS building_name,\n     ...
7    ```sql\nSELECT b.name AS building_name,\n     ...
8    ```sql\nSELECT b.name AS building_name,\n     ...
9    ```sql\nSELECT b.name AS building_name,\n     ...
Name: sql_parsed, dtype: object


In [15]:
import os
import pandas as pd
import openai
from copy import deepcopy
from openai import OpenAI
# Configura tu API Key
openai.api_key = os.getenv("OPENAI_API_KEY")  # o pon aquí tu clave directamente: openai.api_key = "tu_api_key"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def review_question_sql(batch):
    """
    Usa la API de OpenAI para generar n parafraseos de una pregunta dada.
    """
    questions=''
    for idx, row in batch.iterrows():
        questions+=(
            f"{idx+1}. User Question: {row['question']}\n   SQL Query: {row['sql_parsed']}\n"
        )
    prompt = f"""{questions}

Instrucciones:

Valida si la consulta SQL responde correctamente y completamente a la pregunta del usuario.

Indica cualquier error o mejora, tanto en la lógica de la consulta como en la forma en que se extraen los datos.

Señala si faltan columnas relevantes, si se consulta información innecesaria, o si hay errores de sintaxis o incompatibilidad con el esquema.

Si la consulta es correcta y eficiente, responde simplemente: "Correcta".

Si hay problemas, señala exactamente cuáles, justifica tu respuesta y sugiere una versión mejorada del SQL.

Devuelve una lista de evaluaciones en el mismo orden. Si una consulta es correcta, solo pon "Correcta". Si no, da feedback concreto y una versión corregida.

Formato respuesta:
1. Correcta.
2. Correcta.
3. Incorrecta, razon de por que es incorrecta (si es que es incorrecta).
...

"""
    
    try:
        response = client.responses.create(
            model="gpt-4.1-nano",
            instructions = """Eres un experto en bases de datos y procesamiento de lenguaje natural. Tu tarea es revisar la correspondencia entre una pregunta de usuario (user question) y una consulta SQL (sql query) generada automáticamente, asegurándote de lo siguiente:

Corrección sintáctica: La consulta SQL debe estar correctamente escrita y sin errores de sintaxis.

Ajuste al esquema: Debe utilizar únicamente las tablas y columnas que existen en el esquema proporcionado, siguiendo sus restricciones y relaciones.

Relevancia: La consulta SQL debe extraer toda la información necesaria (y sólo la necesaria) para poder responder con precisión a la pregunta del usuario.

Eficiencia: Cuando sea posible, la consulta debe evitar operaciones innecesarias o costosas (por ejemplo, evitar SELECT * si no es necesario, evitar subconsultas redundantes, etc.).

Esquema:

CREATE TABLE smart_buildings.building (
    cups TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    address TEXT,
    type TEXT
);

CREATE TABLE smart_buildings.energy_consumption_monthly_metrics (
    cups TEXT NOT NULL,
    year_month DATE NOT NULL,
    total_consumption_kwh DOUBLE PRECISION,
    avg_daily_consumption_kwh DOUBLE PRECISION,
    total_consumption_prev_month_kwh DOUBLE PRECISION,
    diff_pct_consumption_prev_month DOUBLE PRECISION,
    std_daily_consumption_kwh DOUBLE PRECISION,
    ytd_consumption_kwh NUMERIC,
    ytd_prev_year_consumption_kwh NUMERIC,
    total_consumption_prev_year_same_month_kwh NUMERIC,
    date_insert TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (cups, year_month),
    FOREIGN KEY (cups) REFERENCES smart_buildings.building(cups)
);

CREATE TABLE smart_buildings.energy_consumption_weekly_metrics (
    cups TEXT NOT NULL,
    week_start DATE NOT NULL,
    total_consumption_kwh DOUBLE PRECISION,
    daily_consumption_kwh DOUBLE PRECISION,
    total_consumption_prev_week_kwh DOUBLE PRECISION,
    diff_pct_consumption_prev_week DOUBLE PRECISION,
    std_daily_consumption_kwh DOUBLE PRECISION,
    date_insert TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (cups, week_start),
    FOREIGN KEY (cups) REFERENCES smart_buildings.building(cups)
);""",
            input = prompt,
        )
        text = response.output_text
        # Asumimos que el modelo devuelve las preguntas parafraseadas una por línea
        # En caso de que genere más o menos, tomamos solo n
        return text
    except Exception as e:
        print(f"Error al parafrasear: {e}")
        return ''

results = []
for i in range(0, len(df), 10):
    response = review_question_sql(batch)

    # Divide la respuesta en líneas
    evaluations = response.strip().split('\n')
    # Si hay líneas en blanco, filtra
    evaluations = [line.strip() for line in evaluations if line.strip()]
    # Asume que el orden se mantiene
    results.extend(evaluations)

# Agrega los resultados al dataframe
df['evaluation'] = results
df.to_csv('questions_sql_evaluated.csv', index=False)



ValueError: Length of values (830) does not match length of index (827)

In [18]:
print(results)

['1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '3. Correcta.', '4. Correcta.', '5. Correcta.', '6. Correcta.', '7. Correcta.', '8. Correcta.', '9. Correcta.', '10. Correcta.', '1. Correcta.', '2. Correcta.', '

In [5]:
import pandas as pd

df = pd.read_csv("training.csv",sep=";")
df.head()

,question,type,sql,building names,variation,current month,schema,columna desc
0,Cual es el edificio con mayor consumo en abril...,Consumo Total Mensual,"SELECT b.name AS building_name, m.year_month, ...","['nombre P', 'nombre R']",original,CURRENT MONTH (yyyy-MM-01): 2024-12-01,CREATE TABLE smart_buildings.building (\n c...,## Table: building\n\n* **cups**: Universal Su...
1,¿Qué edificio tuvo el consumo más alto en abri...,Consumo Total Mensual,"SELECT b.name AS building_name, m.year_month, ...","['nombre G', 'nombre D']",rephrased_1,CURRENT MONTH (yyyy-MM-01): 2024-12-01,CREATE TABLE smart_buildings.building (\n c...,## Table: building\n\n* **cups**: Universal Su...
2,Cual es el edifício que mas consumio en abril ...,Consumo Total Mensual,"SELECT b.name AS building_name, m.year_month, ...","['nombre W', 'nombre B']",rephrased_2,CURRENT MONTH (yyyy-MM-01): 2024-12-01,CREATE TABLE smart_buildings.building (\n c...,## Table: building\n\n* **cups**: Universal Su...
3,Me podrías decir cuál edificio consumió más en...,Consumo Total Mensual,"SELECT b.name AS building_name, m.year_month, ...","['nombre Z', 'nombre O']",rephrased_3,CURRENT MONTH (yyyy-MM-01): 2024-12-01,CREATE TABLE smart_buildings.building (\n c...,## Table: building\n\n* **cups**: Universal Su...
4,"Entre todos los edificios, ¿cuál fue el que co...",Consumo Total Mensual,"SELECT b.name AS building_name, m.year_month, ...","['nombre A', 'nombre F']",rephrased_4,CURRENT MONTH (yyyy-MM-01): 2024-12-01,CREATE TABLE smart_buildings.building (\n c...,## Table: building\n\n* **cups**: Universal Su...


In [9]:
import pandas as pd
from openai import OpenAI
import time
import json
# Initialize OpenAI client
client = OpenAI()

# Example DataFrame setup
# Replace this with your actual DataFrame loading step

def build_batch_prompt(questions: list, sql_queries: list,schema:str,column_description:str,current_month:str) -> str:
    """
    Sends the question and sql_query to the LLM to get correctness review.
    Returns the model's response as a string.
    """
    inputs=""
    for i, (q, s) in enumerate(zip(questions, sql_queries), 1):
        inputs += f"{i}. Question: {q}\n   SQL Query: {s}\n"
    prompt = f"""You are an expert SQL reviewer.

Postgres SQL Schema:
{schema}

Column Descriptions:
{column_description}

Current Month: {current_month}

- Check if the SQL syntax is correct.
- Evaluate whether the query is efficient and optimized (e.g., uses appropriate columns, filters, joins, and avoids unnecessary operations).
- Determine if the query accurately answers the user's question.
- Identify any missing conditions, incorrect table or column usage, or logic errors.

Return a JSON array, one object per input
- correct: true or false
- comments: brief explanation of any issues. leave as "none" if it is correct.

Example output:
{{"correct": false, "comments": "The query has this problem..."}}
Input pairs:
{inputs}

"""
    return prompt

results=[]
batch_size = 10
for start in range(0, len(df), batch_size):
    end = min(start + batch_size, len(df))
    batch = df.iloc[start:end]
    prompt = build_batch_prompt(batch["question"], batch["sql"],batch["schema"],batch["columna desc"],batch["current month"])

    try:
        response = client.responses.create(
            model="gpt-4.1-mini",
            input=prompt
        )
        evals = json.loads(response.output_text.strip())
        # In case the output is not exactly a JSON array, you may need to parse/fix the response.
        if isinstance(evals, dict):
            evals = [evals]
        for ev in evals:
            results.append(ev)
    except Exception as e:
        print(e)


df["review"] = [json.dumps(r) for r in results]

# Show the DataFrame with appended reviews
print(df)


Expecting value: line 12 column 17 (char 728)
Expecting value: line 1 column 1 (char 0)


ValueError: Length of values (827) does not match length of index (847)

In [10]:
results

[{'correct': True,
  'comments': 'The query syntax is correct, it properly joins building and monthly consumption metrics using cups, filters by the right month (April 2024), orders by total_consumption_kwh descending to get the building with the highest consumption, and calculates the consumption difference correctly.'},
 {'correct': True,
  'comments': "The query accurately fetches the building with the highest consumption in April 2024, along with the comparison to the previous month's consumption, matching the user question."},
 {'correct': True,
  'comments': 'This query is syntactically and logically correct to find the building with the highest consumption in April and the change from the previous month.'},
 {'correct': True,
  'comments': 'The query is correct and efficient, correctly answering which building consumed the most energy in April and how consumption changed from the previous month.'},
 {'correct': True,
  'comments': 'Correct and optimized query that returns the bu

In [20]:
parsed = sqlparse.parse("""SELECT
  SUM(m.total_consumption_kwh)
FROM smart_buildings.building b
JOIN smart_buildings.energy_consumption_monthly_metrics m
  ON b.cups = m.cups
WHERE
  b.type = 'Educación'
ORDER BY
  SUM(m.total_consumption_kwh)
LIMIT 1;""")

if not parsed or not parsed[0].tokens:
    print('ok')